# Фреймворк PyTorch для разработки искусственных нейронных сетей

## Урок 5. Face Detection and Emotion Recognition (Курсовая работа)

Нужно написать приложение, которое будет считывать и выводить кадры с веб-камеры. В процессе считывания определять что перед камерой находится человек, задетектировав его лицо на кадре. После этого, человек показывает жесты руками, а алгоритм должен считать их и определенным образом реагировать на эти жесты.
На то, как система будет реагировать на определенные жесты - выбор за вами. Например, на определенный жест (жест пис), система будет здороваться с человеком. На другой, будет делать скриншот экрана. И т.д.
Для распознавания жестов, вам надо будет скачать датасет https://www.kaggle.com/gti-upm/leapgestrecog, разработать модель для обучения и обучить эту модель.

*(Усложненное задание) Все тоже самое, но воспользоваться этим датасетом:
https://fitnessallyapp.com/datasets/jester/v1

# Часть 2. Использование модели

In [34]:
import os
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as tt
import matplotlib.pyplot as plt

from PIL import Image

In [35]:
os.environ['KMP_DUPLICATE_LIB_OK']='True' # Для обхода бага библиотек

In [ ]:
class ResNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = self.conv_block(in_channels, 64)
        self.conv2 = self.conv_block(64, 64, pool=True)
        self.res1 = nn.Sequential(self.conv_block(64, 64), self.conv_block(64, 64))
        self.drop1 = nn.Dropout(0.5)
        
        self.conv3 = self.conv_block(64, 128)
        self.conv4 = self.conv_block(128, 128, pool=True)
        self.res2 = nn.Sequential(self.conv_block(128, 128), self.conv_block(128, 128))
        self.drop2 = nn.Dropout(0.5)
        
        self.conv5 = self.conv_block(128, 256)
        self.conv6 = self.conv_block(256, 256, pool=True)
        self.res3 = nn.Sequential(self.conv_block(256, 256), self.conv_block(256, 256))
        self.drop3 = nn.Dropout(0.5)
        
        self.classifier = nn.Sequential(nn.MaxPool2d(6), 
                                        nn.Flatten(),
                                        nn.Linear(256, num_classes))
    
    @staticmethod
    def conv_block(in_channels, out_channels, pool=False):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
                  nn.BatchNorm2d(out_channels), 
                  nn.ELU(inplace=True)]
        if pool: layers.append(nn.MaxPool2d(2))
        return nn.Sequential(*layers)
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.drop1(out)
        
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.drop2(out)
        
        out = self.conv5(out)
        out = self.conv6(out)
        out = self.res3(out) + out
        out = self.drop3(out)
        
        out = self.classifier(out)
        return out

In [46]:
# Класс детектирования и обработки жестов с веб-камеры 
class GestureDetector(object):

    def __init__(self):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = ResNet(1, 10).to(self.device)
        self.model.load_state_dict(torch.load('./models/model1.pth'))
        self.model.eval()

    # Функция рисования найденных параметров на кадре
    def _draw(self, frame, text):
        try:
            # пишем на кадре какой жест распознан
            cv2.putText(frame, text, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        except:
            print('Something wrong im draw function!')

        return frame
    
    @staticmethod
    def digit_to_classname(digit):
        if digit == 0:
            return 'palm'
        elif digit == 1:
            return 'I'
        elif digit == 2:
            return 'fist'
        elif digit == 3:
            return 'fist_moved'
        elif digit == 4:
            return 'thumb'
        elif digit == 5:
            return 'index'
        elif digit == 6:
            return 'ok'
        elif digit == 7:
            return 'palm_moved'
        elif digit == 8:
            return 'c'
        elif digit == 9:
            return 'down'

    # Функция в которой будет происходить процесс считывания и обработки каждого кадра
    def run(self):              
        # Заходим в бесконечный цикл
        while True:
            # Считываем каждый новый кадр - frame
            # ret - логическая переменая. Смысл - считали ли мы кадр с потока или нет
            ret, frame = cap.read()
            #try:
            transforms = tt.Compose([# Настройки для расширения датасета
                         tt.Grayscale(num_output_channels=1), # Картинки чернобелые
                         tt.Resize((64,64), interpolation=Image.BILINEAR),
                         tt.ToTensor(),
                         tt.Normalize((0.1307,), (0.3081,)),
            ])                      # Приведение к тензору
            gesture = transforms(Image.fromarray(frame))
            # Меняем размер изображения для входа в нейронную сеть
            #gesture = cv2.resize(frame,(64,64))
            # Превращаем в 1-канальное серое изображение
            #gesture = cv2.cvtColor(gesture, cv2.COLOR_BGR2GRAY)
            # Превращаем numpy-картинку в pytorch-тензор
            #torch_gesture = torch.from_numpy(gesture).unsqueeze(0).to(self.device).float()
            torch_gesture = gesture.unsqueeze(0).to(self.device).float()
            predict = self.model(torch_gesture.unsqueeze(0)[0])
            # Интерпретируем предсказание как строку нашей эмоции
            predict_text = self.digit_to_classname(predict.argmax())
            # Рисуем на кадре
            self._draw(frame, predict_text)
            #self._draw(gesture, predict_text)
            #plt.imshow(Image.fromarray(gesture))
            #plt.imshow(gesture[0])

            #except:
            #    print('Something wrong im main cycle!')

            # Показываем кадр в окне, и называем его(окно) - 'Gesture Detection'
            cv2.imshow('Gesture Detection', frame)
            
            # Функция, которая проверяет нажатие на клавишу 'q'
            # Если нажатие произошло - выход из цикла. Конец работы приложения
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        # Очищаем все объекты opencv, что мы создали
        cap.release()
        cv2.destroyAllWindows()
        
        
# Создаем объект для считывания потока с веб-камеры(обычно вебкамера идет под номером 0. иногда 1)
cap = cv2.VideoCapture(0)  
# Создаем объект нашего класса приложения
fcd = GestureDetector()
# Запускаем
fcd.run()